In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Load the dataset
file_path = '/content/datagov_fy25_q2 .csv'
try:
    df = pd.read_csv(file_path, encoding='utf-8')
except UnicodeDecodeError:
    try:
        df = pd.read_csv(file_path, encoding='latin-1')
    except Exception as e:
        print(f"Could not read the file with utf-8 or latin-1 encoding: {e}")
        exit()


# Clean column names by stripping whitespace
df.columns = df.columns.str.strip()

# Convert relevant columns to numeric, coercing errors to NaN
numeric_cols = [
    'Approved/Declined Amount',
    'Disbursed/Shipped Amount',
    'Undisbursed Exposure Amount',
    'Outstanding Exposure Amount',
    'Small Business Authorized Amount',
    'Woman Owned Authorized Amount',
    'Minority Owned Authorized Amount'
]

for col in numeric_cols:
    df[col] = df[col].astype(str).str.replace(',', '').str.replace('$', '').str.strip()
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Fill NaN numeric values with 0 for visualization purposes if appropriate, or drop rows if NaNs are problematic
df[numeric_cols] = df[numeric_cols].fillna(0)

# --- 1. Scatterplot Matrix ---
# Selecting a subset of numerical columns for the scatterplot matrix due to the large number of numeric columns
# and to ensure readability.
# Let's pick some key financial measures.
scatter_matrix_cols = [
    'Approved/Declined Amount',
    'Disbursed/Shipped Amount',
    'Outstanding Exposure Amount',
    'Small Business Authorized Amount'
]

# Filter out rows where all selected columns are zero to make the plot more meaningful if many are zero
df_filtered_scatter = df[df[scatter_matrix_cols].sum(axis=1) > 0].copy()

if not df_filtered_scatter.empty:
    print("Generating Scatterplot Matrix...")
    sns.pairplot(df_filtered_scatter[scatter_matrix_cols])
    plt.suptitle('Scatterplot Matrix of Financial Measures', y=1.02) # Adjust suptitle position
    plt.show()
else:
    print("Not enough non-zero data for Scatterplot Matrix after filtering.")

# --- 2. Parallel Coordinates Plot ---
# Using a subset of numerical columns for clarity.
# Parallel coordinates work best with normalized data for better comparison across different scales.
# parallel_coords_cols = [
#     'Approved/Declined Amount',
#     'Disbursed/Shipped Amount',
#     'Outstanding Exposure Amount',
#     'Small Business Authorized Amount',
#     'Woman Owned Authorized Amount',
#     'Minority Owned Authorized Amount'
# ]

# # Filter out rows where all selected columns are zero for parallel coordinates
# df_filtered_parallel = df[df[parallel_coords_cols].sum(axis=1) > 0].copy()

# if not df_filtered_parallel.empty:
#     print("Generating Parallel Coordinates Plot...")
#     # Normalize data for better visualization in parallel coordinates
#     df_normalized = df_filtered_parallel[parallel_coords_cols].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
#     # Remove the 'Country' column from df_normalized for the plot
#     # df_normalized['Country'] = df_filtered_parallel['Country'] # Add a categorical column for color if desired

#     fig_parallel = px.parallel_coordinates(
#         df_normalized,
#         dimensions=parallel_coords_cols,
#         # color="Country", # Color by country to see patterns
#         # color_continuous_scale=px.colors.sequential.Viridis, # Removed color_continuous_scale
#         title="Parallel Coordinates Plot of Financial Measures" # Removed color from title
#     )
#     fig_parallel.show()
# else:
#     print("Not enough non-zero data for Parallel Coordinates Plot after filtering.")


# --- 3. Line Graph ---
# Let's visualize 'Approved/Declined Amount' across different 'Primary Exporter State Name'
# We'll aggregate the data first (e.g., sum of amounts per state)
df_line_graph = df.groupby('Primary Exporter State Name')['Approved/Declined Amount'].sum().reset_index()
df_line_graph = df_line_graph.sort_values(by='Approved/Declined Amount', ascending=False).head(15) # Top 15 states

if not df_line_graph.empty:
    print("Generating Line Graph...")
    plt.figure(figsize=(12, 6))
    plt.plot(df_line_graph['Primary Exporter State Name'], df_line_graph['Approved/Declined Amount'], marker='o')
    plt.title('Total Approved/Declined Amount by Top 15 Primary Exporter States')
    plt.xlabel('Primary Exporter State Name')
    plt.ylabel('Total Approved/Declined Amount')
    plt.xticks(rotation=45, ha='right')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()
else:
    print("Not enough data for Line Graph after aggregation.")

# --- 4. Stacked Bar Chart ---
# Let's visualize 'Approved/Declined Amount' by 'Program' and 'Policy Type'
# Aggregate data: sum of 'Approved/Declined Amount' for each combination of 'Program' and 'Policy Type'
df_stacked_bar = df.groupby(['Program', 'Policy Type'])['Approved/Declined Amount'].sum().unstack(fill_value=0)

# Select top N programs for better visualization if there are too many
top_programs = df.groupby('Program')['Approved/Declined Amount'].sum().nlargest(10).index
# Filter df_stacked_bar to include only the top programs
df_stacked_bar_filtered = df_stacked_bar[df_stacked_bar.index.isin(top_programs)]


if not df_stacked_bar_filtered.empty:
    print("Generating Stacked Bar Chart...")
    df_stacked_bar_filtered.plot(kind='bar', stacked=True, figsize=(14, 8))
    plt.title('Approved/Declined Amount by Program and Policy Type (Top 10 Programs)')
    plt.xlabel('Program')
    plt.ylabel('Total Approved/Declined Amount')
    plt.xticks(rotation=45, ha='right')
    plt.legend(title='Policy Type', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()
else:
    print("Not enough data for Stacked Bar Chart after aggregation and filtering.")

/tmp/ipython-input-1582000784.py:12: DtypeWarning: Columns (0,13,27,28,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='latin-1')


Generating Scatterplot Matrix...


In [ ]:
print(df.columns)